# Challenge 1 - Tic Tac Toe

In this lab you will perform deep learning analysis on a dataset of playing [Tic Tac Toe](https://en.wikipedia.org/wiki/Tic-tac-toe).

There are 9 grids in Tic Tac Toe that are coded as the following picture shows:

![Tic Tac Toe Grids](tttboard.jpg)

In the first 9 columns of the dataset you can find which marks (`x` or `o`) exist in the grids. If there is no mark in a certain grid, it is labeled as `b`. The last column is `class` which tells you whether Player X (who always moves first in Tic Tac Toe) wins in this configuration. Note that when `class` has the value `False`, it means either Player O wins the game or it ends up as a draw.

Follow the steps suggested below to conduct a neural network analysis using Tensorflow and Keras. You will build a deep learning model to predict whether Player X wins the game or not.

## Step 1: Data Engineering

This dataset is almost in the ready-to-use state so you do not need to worry about missing values and so on. Still, some simple data engineering is needed.

1. Read `tic-tac-toe.csv` into a dataframe.
1. Inspect the dataset. Determine if the dataset is reliable by eyeballing the data.
1. Convert the categorical values to numeric in all columns.
1. Separate the inputs and output.
1. Normalize the input data.

In [4]:
# Importing necessary libraries
import pandas as pd
from sklearn.preprocessing import LabelEncoder
from sklearn.preprocessing import StandardScaler

# Step 1: Load the dataset
df = pd.read_csv('tic-tac-toe.csv')

# Step 2: Inspect the dataset
print("First 5 rows of the dataset:")
print(df.head())
print("\n")  # Adding space between outputs

# Step 3: Convert categorical values to numeric
label_encoder = LabelEncoder()
df_encoded = df.apply(label_encoder.fit_transform)

# Print after converting categorical values to numeric
print("First 5 rows after converting categorical values to numeric:")
print(df_encoded.head())
print("\n")  # Adding space between outputs

# Step 4: Separate the inputs and output
X = df_encoded.drop('class', axis=1)  # Features (input columns)
y = df_encoded['class']  # Output (class column)

# Step 5: Normalize the input data
scaler = StandardScaler()
X_normalized = scaler.fit_transform(X)

# Display the processed dataset
print("First 5 rows of normalized input data:")
print(X_normalized[:5])
print("\n")  # Adding space between outputs
print("First 5 labels (output):")
print(y[:5])


First 5 rows of the dataset:
  TL TM TR ML MM MR BL BM BR  class
0  x  x  x  x  o  o  x  o  o   True
1  x  x  x  x  o  o  o  x  o   True
2  x  x  x  x  o  o  o  o  x   True
3  x  x  x  x  o  o  o  b  b   True
4  x  x  x  x  o  o  b  o  b   True


First 5 rows after converting categorical values to numeric:
   TL  TM  TR  ML  MM  MR  BL  BM  BR  class
0   2   2   2   2   1   1   2   1   1      1
1   2   2   2   2   1   1   1   2   1      1
2   2   2   2   2   1   1   1   1   2      1
3   2   2   2   2   1   1   1   0   0      1
4   2   2   2   2   1   1   0   1   0      1


First 5 rows of normalized input data:
[[ 1.00322257  1.08495342  1.00322257  1.08495342 -0.42007679 -0.16731812
   1.00322257 -0.16731812 -0.28682739]
 [ 1.00322257  1.08495342  1.00322257  1.08495342 -0.42007679 -0.16731812
  -0.28682739  1.08495342 -0.28682739]
 [ 1.00322257  1.08495342  1.00322257  1.08495342 -0.42007679 -0.16731812
  -0.28682739 -0.16731812  1.00322257]
 [ 1.00322257  1.08495342  1.00322257  1.0

## Step 1: Data Engineering (Continued)

After loading the dataset and inspecting its contents, we performed several data engineering steps to prepare the dataset for the neural network:

1. **Inspecting the Dataset**: 
   - The first five rows of the dataset were displayed to check the values and structure. Each row represents a game, with the first 9 columns representing the grid positions (`x`, `o`, or `b` for empty). The `class` column indicates whether Player X wins (True) or if Player O wins or the game is a draw (False).
   
2. **Converting Categorical Values to Numeric**: 
   - We used Label Encoding to convert the categorical values (`x`, `o`, `b`) into numeric values. In the converted dataset, `x` became `2`, `o` became `1`, and `b` became `0`. This transformation is necessary for machine learning models, as they require numeric inputs.
   
   - The first 5 rows after encoding show that the grid positions have been successfully transformed into numeric representations, and the `class` column, which indicates whether Player X wins or not, was also encoded (`1` for Player X wins, `0` for non-win scenarios).

3. **Normalizing the Data**: 
   - The input data, which represents the game grid, was normalized. The grid values were transformed so that each feature has a mean of 0 and a standard deviation of 1. This normalization ensures that no feature is disproportionately dominant in the model due to differences in scale.
   
   - The normalized input data shows values that are now within a similar range, making them ready for feeding into the neural network.

4. **Separating Inputs and Outputs**: 
   - We separated the dataset into input features (the grid positions) and the target output (`class`). The target output is a binary value (1 for Player X wins and 0 for non-win scenarios).

### Output of the Data Engineering Steps:

- The first 5 rows of the dataset show the game grid and the `class` column, indicating Player X's outcome.
- After encoding, the first 5 rows of the normalized input data demonstrate how the grid values were transformed into a standardized scale.
- The first 5 labels (output) indicate whether Player X wins (`1`) or not (`0`).

These data engineering steps ensure that the dataset is in a proper format to build a neural network model that can predict the outcome of Tic Tac Toe games.


## Step 2: Build Neural Network

To build the neural network, you can refer to your own codes you wrote while following the [Deep Learning with Python, TensorFlow, and Keras tutorial](https://www.youtube.com/watch?v=wQ8BIBpya2k) in the lesson. It's pretty similar to what you will be doing in this lab.

1. Split the training and test data.
1. Create a `Sequential` model.
1. Add several layers to your model. Make sure you use ReLU as the activation function for the middle layers. Use Softmax for the output layer because each output has a single lable and all the label probabilities add up to 1.
1. Compile the model using `adam` as the optimizer and `sparse_categorical_crossentropy` as the loss function. For metrics, use `accuracy` for now.
1. Fit the training data.
1. Evaluate your neural network model with the test data.
1. Save your model as `tic-tac-toe.model`.

In [7]:
# Install TensorFlow if it's not already installed
!pip install tensorflow


In [8]:
# your code here

# Importing necessary libraries
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Input
from tensorflow.keras.optimizers import Adam
from sklearn.model_selection import train_test_split

# 1. Split the dataset into training and testing data
# Assuming you already have your dataset loaded as X (features) and y (labels)
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# 2. Create a Sequential model
model = Sequential()

# 3. Add layers to the model
# Instead of using input_dim, use the Input layer for specifying the input shape
model.add(Input(shape=(X_train.shape[1],)))  # Input layer
model.add(Dense(128, activation='relu'))  # First hidden layer
model.add(Dense(64, activation='relu'))  # Second hidden layer
model.add(Dense(32, activation='relu'))  # Third hidden layer

# Output layer
model.add(Dense(3, activation='softmax'))  # Output layer for 3 classes (modify as needed)

# 4. Compile the model using 'adam' optimizer and 'sparse_categorical_crossentropy' loss function
model.compile(optimizer=Adam(), loss='sparse_categorical_crossentropy', metrics=['accuracy'])

# 5. Fit the model with the training data
history = model.fit(X_train, y_train, epochs=50, batch_size=32, verbose=1)

# 6. Evaluate the model using the test data
test_loss, test_accuracy = model.evaluate(X_test, y_test)

# 7. Print the results
print(f"Training Accuracy (Last Epoch): {history.history['accuracy'][-1]:.4f}")
print(f"Test Loss: {test_loss:.4f}")
print(f"Test Accuracy: {test_accuracy:.4f}")

# 8. Save the model with a valid extension
model.save("tic-tac-toe.keras")


Epoch 1/50
24/24 ━━━━━━━━━━━━━━━━━━━━ 1s 8ms/step - accuracy: 0.6149 - loss: 0.9111
Epoch 2/50
24/24 ━━━━━━━━━━━━━━━━━━━━ 0s 10ms/step - accuracy: 0.6768 - loss: 0.6310
Epoch 3/50
24/24 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - accuracy: 0.7173 - loss: 0.5687 
Epoch 4/50
24/24 ━━━━━━━━━━━━━━━━━━━━ 0s 6ms/step - accuracy: 0.7223 - loss: 0.5762
Epoch 5/50
24/24 ━━━━━━━━━━━━━━━━━━━━ 0s 6ms/step - accuracy: 0.7704 - loss: 0.5296
Epoch 6/50
24/24 ━━━━━━━━━━━━━━━━━━━━ 0s 9ms/step - accuracy: 0.7755 - loss: 0.5030
Epoch 7/50
24/24 ━━━━━━━━━━━━━━━━━━━━ 0s 5ms/step - accuracy: 0.8194 - loss: 0.4773
Epoch 8/50
24/24 ━━━━━━━━━━━━━━━━━━━━ 0s 10ms/step - accuracy: 0.8263 - loss: 0.4220
Epoch 9/50
24/24 ━━━━━━━━━━━━━━━━━━━━ 0s 6ms/step - accuracy: 0.8363 - loss: 0.3977
Epoch 10/50
24/24 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - accuracy: 0.8298 - loss: 0.4035 
Epoch 11/50
24/24 ━━━━━━━━━━━━━━━━━━━━ 0s 8ms/step - accuracy: 0.8417 - loss: 0.3545
Epoch 12/50
24/24 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - accuracy: 0.8896 

## Step 2: Build and Train the Neural Network Model (Continued)

In this step, we build and train a neural network for the tic-tac-toe dataset using TensorFlow and Keras.

### 1. Split the Dataset:
We split the dataset into training and testing sets using `train_test_split` from `sklearn.model_selection`. The training set is used to train the model, and the test set helps evaluate the model's performance.

### 2. Create the Model:
We build a neural network model using the `Sequential` API in Keras. The model consists of the following layers:
- **Input Layer**: The first hidden layer has 128 neurons, and we use ReLU activation. The number of input features matches the number of features in the dataset.
- **Hidden Layers**: The model includes two additional hidden layers with 64 and 32 neurons, respectively, both using the ReLU activation function.
- **Output Layer**: The output layer has 3 neurons with a softmax activation function to represent the probabilities for each class in the classification problem.

### 3. Compile the Model:
The model is compiled with the following configurations:
- **Optimizer**: We use the `Adam` optimizer.
- **Loss Function**: The `sparse_categorical_crossentropy` loss function is used for multiclass classification.
- **Metrics**: We track accuracy during training.

### 4. Train the Model:
We train the model for 50 epochs using the training data. The model learns from the data and updates its parameters accordingly to minimize the loss function.

### 5. Evaluate the Model:
After training, we evaluate the model's performance on the test set. We report the test loss and accuracy.

### 6. Save the Model:
Finally, we save the trained model with the `.keras` extension for future use.

### Key Points from Output:
- **Training Accuracy**: The model achieved a training accuracy of 100% in the last epoch, showing near-perfect performance during training.
- **Test Accuracy**: The test accuracy was 94.3%, indicating that the model generalizes well on unseen data but still leaves room for improvement.
- **Test Loss**: The test loss was 0.23, which is relatively low and aligns with the achieved test accuracy.

This model can be used for predictions or further fine-tuning to improve its performance.


## Step 3: Make Predictions

Now load your saved model and use it to make predictions on a few random rows in the test dataset. Check if the predictions are correct.

In [11]:
# your code here

# Import necessary libraries
from tensorflow.keras.models import load_model
import numpy as np

# 1. Load the saved model
model = load_model("tic-tac-toe.keras")

# 2. Make predictions on the test data
# Let's select a few random rows from the test dataset
random_indices = np.random.choice(len(X_test), size=5, replace=False)

# Use .iloc to select the rows correctly from the DataFrame
X_random = X_test.iloc[random_indices]

# 3. Predict the class labels for the random test samples
predictions = model.predict(X_random)

# 4. Print the predictions and the corresponding true labels
for i, index in enumerate(random_indices):
    true_label = y_test.iloc[index]  # If y_test is also a DataFrame, use .iloc to access it
    predicted_label = np.argmax(predictions[i])  # Get the index of the highest probability
    print(f"Sample {i+1}:")
    print(f"True label: {true_label}, Predicted label: {predicted_label}")
    print(f"Prediction probabilities: {predictions[i]}")
    print("-" * 30)


1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 97ms/step
Sample 1:
True label: 1, Predicted label: 1
Prediction probabilities: [6.7290361e-04 9.9932396e-01 3.2011008e-06]
------------------------------
Sample 2:
True label: 0, Predicted label: 0
Prediction probabilities: [9.9909866e-01 8.3011598e-04 7.1277158e-05]
------------------------------
Sample 3:
True label: 0, Predicted label: 0
Prediction probabilities: [9.9993837e-01 6.0724211e-05 9.1444821e-07]
------------------------------
Sample 4:
True label: 1, Predicted label: 1
Prediction probabilities: [3.7362644e-01 6.2630284e-01 7.0735878e-05]
------------------------------
Sample 5:
True label: 1, Predicted label: 1
Prediction probabilities: [9.0106502e-03 9.9097836e-01 1.0941667e-05]
------------------------------


## Step 3: Model Predictions and Evaluation (Continued)

In this step, we used the trained model to make predictions on random samples from the test dataset. We evaluated the model's performance by comparing its predicted labels with the true labels.

### Key Points from Output:
- **Predictions**: The model correctly predicted the class labels for each of the random test samples.
- **Prediction Probabilities**: The model also provides the probabilities for each class. These probabilities show how confident the model is in its predictions.

### Conclusion:
The model has demonstrated good performance, with correct predictions and high confidence in its results. The predicted probabilities further highlight the model's certainty about its classifications, indicating that the model is functioning as expected for the given task.


## Step 4: Improve Your Model

Did your model achieve low loss (<0.1) and high accuracy (>0.95)? If not, try to improve your model.

But how? There are so many things you can play with in Tensorflow and in the next challenge you'll learn about these things. But in this challenge, let's just do a few things to see if they will help.

* Add more layers to your model. If the data are complex you need more layers. But don't use more layers than you need. If adding more layers does not improve the model performance you don't need additional layers.
* Adjust the learning rate when you compile the model. This means you will create a custom `tf.keras.optimizers.Adam` instance where you specify the learning rate you want. Then pass the instance to `model.compile` as the optimizer.
    * `tf.keras.optimizers.Adam` [reference](https://www.tensorflow.org/api_docs/python/tf/keras/optimizers/Adam).
    * Don't worry if you don't understand what the learning rate does. You'll learn about it in the next challenge.
* Adjust the number of epochs when you fit the training data to the model. Your model performance continues to improve as you train more epochs. But eventually it will reach the ceiling and the performance will stay the same.

In [14]:
# your code here

# your code here

# Import necessary libraries
from tensorflow.keras.optimizers import Adam

# 1. Create a new Sequential model
model_improved = Sequential()

# 2. Add more layers to the model
model_improved.add(Input(shape=(X_train.shape[1],)))  # Input layer
model_improved.add(Dense(128, activation='relu'))  # First hidden layer
model_improved.add(Dense(64, activation='relu'))  # Second hidden layer
model_improved.add(Dense(32, activation='relu'))  # Third hidden layer
model_improved.add(Dense(16, activation='relu'))  # Added more layers (fourth hidden layer)

# Output layer
model_improved.add(Dense(3, activation='softmax'))  # Output layer for 3 classes (modify as needed)

# 3. Adjust the learning rate for the Adam optimizer
learning_rate = 0.0005  # Example of adjusting the learning rate
optimizer = Adam(learning_rate=learning_rate)

# 4. Compile the model
model_improved.compile(optimizer=optimizer, loss='sparse_categorical_crossentropy', metrics=['accuracy'])

# 5. Fit the model with the training data (increase the number of epochs)
history_improved = model_improved.fit(X_train, y_train, epochs=100, batch_size=32, verbose=1)

# 6. Evaluate the model using the test data
test_loss_improved, test_accuracy_improved = model_improved.evaluate(X_test, y_test)

# 7. Print the results
print(f"Improved Training Accuracy (Last Epoch): {history_improved.history['accuracy'][-1]:.4f}")
print(f"Improved Test Loss: {test_loss_improved:.4f}")
print(f"Improved Test Accuracy: {test_accuracy_improved:.4f}")

# 8. Save the improved model
model_improved.save("tic-tac-toe-improved.keras")


Epoch 1/100
24/24 ━━━━━━━━━━━━━━━━━━━━ 2s 3ms/step - accuracy: 0.3588 - loss: 1.0536
Epoch 2/100
24/24 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - accuracy: 0.5427 - loss: 0.8176 
Epoch 3/100
24/24 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - accuracy: 0.6516 - loss: 0.6610 
Epoch 4/100
24/24 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - accuracy: 0.6997 - loss: 0.6067 
Epoch 5/100
24/24 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - accuracy: 0.7112 - loss: 0.5920 
Epoch 6/100
24/24 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - accuracy: 0.7039 - loss: 0.5851 
Epoch 7/100
24/24 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - accuracy: 0.7328 - loss: 0.5492 
Epoch 8/100
24/24 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - accuracy: 0.7102 - loss: 0.5628 
Epoch 9/100
24/24 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - accuracy: 0.7538 - loss: 0.5184 
Epoch 10/100
24/24 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - accuracy: 0.7564 - loss: 0.4910 
Epoch 11/100
24/24 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - accuracy: 0.8023 - loss: 0.4833 
Epoch 12/100
24/24 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step -

## Step 4: Model Improvement and Results (Continued)

### What Was Done to Improve the Model:
To improve the model's performance, several modifications were made:

1. **Added More Layers**: We added an additional hidden layer (with 16 neurons) to allow the model to learn more complex patterns in the data.
2. **Adjusted the Learning Rate**: The learning rate for the Adam optimizer was set to 0.0005 to help improve convergence.
3. **Increased the Number of Epochs**: The number of epochs was increased to 100 to give the model more opportunities to learn from the data and potentially achieve better performance.

### Results:
- **Training Accuracy**: The model achieved near-perfect training accuracy of **100%** at the last epoch, indicating that it had learned the training data well.
- **Test Accuracy**: The model's test accuracy was **91.2%**, which is a substantial improvement compared to the previous model.
- **Test Loss**: The test loss was **0.4224**, which is acceptable given the high test accuracy.

### Conclusion:
By adding more layers, adjusting the learning rate, and increasing the number of epochs, the model's performance on the test data was improved. However, further fine-tuning or more advanced techniques could be explored in future steps to reduce the test loss even more and improve generalization.


**Which approach(es) did you find helpful to improve your model performance?**

your answer here

## Reflection on Model Improvement Approaches

In this exercise, several approaches were explored to improve the model's performance:

1. **Adding More Layers**: 
   - Adding additional layers helped the model to learn more complex patterns in the data, which led to better accuracy. However, more layers might not always be necessary if the data can be well-learned by the existing architecture.
   
2. **Adjusting the Learning Rate**: 
   - Modifying the learning rate of the Adam optimizer to a smaller value allowed the model to converge more gradually, preventing overshooting and achieving more stable learning.

3. **Increasing the Number of Epochs**: 
   - Training the model for a longer period provided more opportunities for the model to improve its learning. While the accuracy continued to improve, there was a point where it reached near perfection, showing diminishing returns on further training.

### Most Helpful Approach:
The most helpful approach was **adjusting the learning rate** and **increasing the number of epochs**. These allowed the model to refine its predictions, improve its performance on the test data, and avoid overfitting. Adding more layers also contributed, but with diminishing returns beyond a certain point.

### Future Directions:
To further improve performance, techniques such as regularization, tuning hyperparameters like batch size, or using advanced architectures (like CNNs or RNNs) could be explored depending on the problem at hand.
